# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
cities_csv = "./cities.csv" 
cities_df = pd.read_csv(cities_csv)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hermanus,-34.42,19.23,54.00,92,12,8.72,ZA,1611796843
1,Hilo,19.73,-155.09,73.40,78,90,4.61,US,1611796843
2,Punta Arenas,-53.15,-70.92,42.80,87,90,9.22,CL,1611796843
3,Esperance,-33.87,121.90,68.00,68,75,9.22,AU,1611796843
4,Albany,42.60,-73.97,28.99,61,99,1.01,US,1611796843


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)

# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_cities_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0),:].reset_index()
ideal_cities_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,7,Saint-Philippe,-21.36,55.77,77.00,61,0,6.91,RE,1611796844
1,20,Vila Velha,-20.33,-40.29,78.80,73,0,9.22,BR,1611796518
2,156,Presidencia Roque Sáenz Peña,-26.79,-60.44,79.00,75,0,3.00,AR,1611796574
3,167,Niquero,20.04,-77.58,75.00,79,0,3.96,CU,1611796576
4,204,Mumford,5.26,-0.76,78.03,91,0,8.34,GH,1611796583
5,214,Mercedes,-34.65,-59.43,77.00,46,0,3.00,AR,1611796585
6,230,Saint-Joseph,-21.37,55.62,77.00,61,0,6.91,RE,1611796589
7,231,Saint-Pierre,-21.34,55.48,77.00,61,0,6.91,RE,1611796589
8,337,La Plata,-34.92,-57.95,75.00,73,0,1.99,AR,1611796610
9,459,São Filipe,14.90,-24.50,73.38,69,0,5.03,CV,1611796635


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotel_df = ideal_cities_df.loc[:,["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,-21.36,55.77,
1,Vila Velha,BR,-20.33,-40.29,
2,Presidencia Roque Sáenz Peña,AR,-26.79,-60.44,
3,Niquero,CU,20.04,-77.58,
4,Mumford,GH,5.26,-0.76,
5,Mercedes,AR,-34.65,-59.43,
6,Saint-Joseph,RE,-21.37,55.62,
7,Saint-Pierre,RE,-21.34,55.48,
8,La Plata,AR,-34.92,-57.95,
9,São Filipe,CV,14.90,-24.50,


In [56]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "lodging"
radius = "5000"

params = {
    "types": target_search,
    "radius": radius,
    "key": g_key
}


print(f"Beginning of Retrieval")

for index,row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print(f"Not found... skipping")
print(f"End of Retrieval")
    

Beginning of Retrieval
Closest hotel is Chambres d'hôte "La Trinité".
Closest hotel is Hotel Vitória Palace.
Closest hotel is Hotel Aconcagua.
Not found... skipping
Closest hotel is Spanish Villa.
Closest hotel is Posada del Inti Centro.
Closest hotel is "Plantation Bed and Breakfast.
Closest hotel is Lindsey Hôtel.
Closest hotel is Hotel Corregidor.
Closest hotel is Tortuga B&B.
Closest hotel is Hotel San Isidro Oficial.
End of Retrieval


In [57]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
1,Vila Velha,BR,-20.33,-40.29,Hotel Vitória Palace
2,Presidencia Roque Sáenz Peña,AR,-26.79,-60.44,Hotel Aconcagua
3,Niquero,CU,20.04,-77.58,
4,Mumford,GH,5.26,-0.76,Spanish Villa
5,Mercedes,AR,-34.65,-59.43,Posada del Inti Centro
6,Saint-Joseph,RE,-21.37,55.62,"""Plantation Bed and Breakfast"
7,Saint-Pierre,RE,-21.34,55.48,Lindsey Hôtel
8,La Plata,AR,-34.92,-57.95,Hotel Corregidor
9,São Filipe,CV,14.90,-24.50,Tortuga B&B


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [64]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

# Fill NaN values and convert to float
rating = cities_df["Humidity"].astype(float)
general_locations = cities_df[["Lat", "Lng"]]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(general_locations, weights=rating, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(locations, 
    info_box_content=hotel_info)
fig.add_layer(markers)
fig
# # Display figure


# # Display figure


Figure(layout=FigureLayout(height='420px'))